In [1]:
np.set_printoptions(threshold=sys.maxsize)

NameError: name 'np' is not defined

# This is my prototyping platform for the code to extract a timeseries of data from the NLDAS GRB files and store them in NetCDF format.
## There should be a .py script with a similar name that runs the finished code on HPC.

In [2]:
import datetime as dt
import numpy as np
import netCDF4 as nc # http://unidata.github.io/netcdf4-python/
import pygrib as pg
import sys
import os
from tqdm.notebook import trange, tqdm
import pickle as pkl
import nldas_pygrib_tools as npt
import time

In [3]:
nc_year=1980

In [4]:
# Write the NetCDF forcing data file.
grib_dir = '/home/NearingLab/data/nldas/grib/NLDAS2.FORCING/'
write_dir = '/home/NearingLab/data/nldas/netcdf-single-cells/'+str(nc_year)+'/'

In [5]:
# Open an example file
fname = grib_dir + str(nc_year) +'/001/' + 'NLDAS_FORA0125_H.A'+ str(nc_year) +'0101.0000.002.grb'
gbf_temp = pg.open(fname)
lats = gbf_temp[1].latitudes
lons = gbf_temp[1].longitudes
nrows = gbf_temp[11].values.shape[0]
ncols = gbf_temp[11].values.shape[1]
N = nrows*ncols

In [6]:
gvars = {0:'airtemp', 1:'spechum', 2:'airpres', 3:'forcingUGRD', 4:'windspd',
         5:'LWRadAtm',6:'forcingCONVfrac', 7:'forcingCAPE', 8:'forcingPEVAP', 9:'pptrate', 10:'SWRadAtm'}
fvars = {0:'airpres', 1:'airtemp', 2:'pptrate', 3:'spechum', 4:'windspd', 5:'LWRadAtm',6:'SWRadAtm'}

In [7]:
# Set start and end data information for the GRIB/NetCDF forcing data.
yearStart  = nc_year
monthStart = 1 
dayStart   = 1 
hourStart  = 0
startDateTime = dt.datetime(yearStart, monthStart, dayStart, hour = hourStart)
print("Will be calculating hours starting from: ")
print(startDateTime)
dayOfYearStart = dt.datetime.date(startDateTime).timetuple().tm_yday
yearEnd  = nc_year
monthEnd = 12
dayEnd   = 31 
hourEnd  = 23
endDateTime = dt.datetime(yearEnd, monthEnd, dayEnd, hour = hourEnd)
dayOfYearEnd = dt.datetime.date(endDateTime).timetuple().tm_yday

Will be calculating hours starting from: 
1980-01-01 00:00:00


In [8]:
# Initializing the directory, but will change each day and year.
mainDirectory = '/home/NearingLab/data/nldas/grib/NLDAS2.FORCING/'
startDirectory = mainDirectory + str(yearStart)  + "/" \
    + str("{:03d}".format(dayOfYearStart))  + "/"
endDirectory = mainDirectory + str(yearEnd)  + "/" \
    + str("{:03d}".format(dayOfYearEnd))  + "/"
filePrefix = 'NLDAS_FORA0125_H.A'
fileSufix = '.002.grb'

In [9]:
# specify the data and time to fine the correct file in this name format
startFileDateTime = npt.dateForFile(yearStart, monthStart, dayStart, hourStart)
endFileDateTime = npt.dateForFile(yearEnd, monthEnd, dayEnd, hourEnd)
#Add prefix and sufix to the date to create the whole file name.
startFile = npt.getFileName(startFileDateTime, startDirectory, "A")
endFile = npt.getFileName(endFileDateTime, endDirectory, "A")

In [10]:
# Need to get the GRIB time for the first and last files
#Start the loop at the first date in the files.
year1, month1, day1, hour1 = npt.dateFromGRIB(startFile)
t = dt.datetime(year1, month1, day1, hour=hour1)
#Then have the loop run until the last file date.
year2, month2, day2, hour2 = npt.dateFromGRIB(endFile)
endTime = dt.datetime(year2, month2, day2, hour=hour2)
# Set timestep to move forward, to run through the files
deltime = dt.timedelta(hours=1)
# Estimate the number of hours in the record
H = endTime - t # (t = startDateTime)
# Convert the time difference to hours) 
H = int(H.total_seconds()/60/60) + 1
time_series = [0 for x in range(H)]

In [11]:
# Make a list of all the times to loop through
dates = [startDateTime + deltime*h for h in range(H)]
len(dates)

8784

In [12]:
# find all the masked cells before the main loop, and avoid them
ixy = -1 # Start at -1, so when we add the first value before the mask check, it goes to 0
xy_list = []
for x in tqdm(range(464)):
    for y in range(224):            
        ixy+=1 # lat/lon from the 1D arrays that correspond to these indices
        if np.ma.is_masked(gbf_temp[11].values[y, x]): # Skip masked cells, takes .0044 seconds
            continue
        xy_list.append(ixy)

In [13]:
# Set up a dictionary for each cell, keys named from lat-lon
# Will be filled in with data from the grib files
G = {}
for ixy in tqdm(xy_list):
    xy = npt.name_xy(ixy, lats, lons)
    G[xy] = npt.setForcingLists(H)

In [24]:
# Main loop through the GRIB files by one hour intervals. open, extract, write, save
# Main loop through the NetCDF files by one hour intervals. 
# iH: Index to use for filling forcing data list.
for iH, t in enumerate(tqdm(dates)):

    hoursSinceStartDate = t - startDateTime
    hoursSinceStartDate = int(hoursSinceStartDate.total_seconds()/60/60)
    time_series[iH] = float(hoursSinceStartDate)

    # The files have both A and B versions.
    AB = "A"
    # Set the strings for the file name
    iYear, iMonth, iDay, iHour = npt.getValuesFromDateTime(t)
    # Get the datetime stuff in strings to be used in the NetCDF file call.
    dateTime4File = npt.dateForFile(iYear, iMonth, iDay, iHour)
    # Need to change the directory to reflect the loop data
    directory = npt.changeDirectory(t, grib_dir)
    # Put the file name together, this includes the full path
    fileName = npt.getFileName(dateTime4File, directory, AB)
    # Open the file for this particular data & time.
    try:
        gbf = pg.open(fileName)
    except:
        # skip the file
        print('File not found: \n',fileName)
        continue
    
    #####################################################################
    #####   THIS IS the function to actually get the forcing data  ######
    g = npt.extractGrib(gbf, xy_list, nrows, ncols)

    # Looping takes too long. Need to get all values in vector
    # through x,y 1D indices.
    for i, ixy in enumerate(xy_list):

        xy = npt.name_xy(ixy, lats, lons)
        
        # Need to get the two dimensional x,y values from the 1D xy
        x, y = np.unravel_index(ixy, (ncols,nrows))
                
        # Fill in the main Grib dictionary.
        for iv, v in enumerate(fvars):
            G[xy][fvars[v]][iH] = g[fvars[v]][i]

# Save the whole data periodically.
save_G_name = write_dir+'grib_export.pkl'
with open(save_G_name,'wb') as f:
    pkl.dump(G, f)
os.chmod(save_G_name, 0o777)

KeyboardInterrupt: 

In [20]:
xy_list[-10:-1]

[103926, 103927, 103928, 103929, 103930, 103931, 103932, 103933, 103934]

In [14]:
def extractGrib(g, xy_list, nrows, ncols, verbose=False):
    # 1:11:11 TMP, 2-m above ground Temperature [K]
    airtemp = np.array(g[1].values.reshape(nrows*ncols)[xy_list])

    # 2:51:51 SPFH, 2-m above ground Specific humidity [kg/kg]
    spechum = np.array(g[2].values.reshape(nrows*ncols)[xy_list])
    # 3:1:1 PRES, Surface pressure [Pa]
    airpres = np.array(g[3].values.reshape(nrows*ncols)[xy_list])
    # 4:33:33 UGRD, 10-m above ground Zonal wind speed [m/s]
    forcingUGRD = np.array(g[4].values.reshape(nrows*ncols)[xy_list])
    # 5:34:34 VGRD, 10-m above ground Meridonal wind speed [m/s]
    windspd = np.array(g[5].values.reshape(nrows*ncols)[xy_list])
    # 6:205:205 DLWRF,  Longwave radiation flux downwards [W/m^2]
    LWRadAtm = np.array(g[6].values.reshape(nrows*ncols)[xy_list])
    # 7:153:153 CONVfrac, Frac of total precip convective
    forcingCONVfrac = np.array(g[7].values.reshape(nrows*ncols)[xy_list])
    # 8:157:157 CAPE, 180-mb above ground Convective Available Potential Energy
    forcingCAPE = np.array(g[8].values.reshape(nrows*ncols)[xy_list])
        
    # PEVAP, Potential evaporation hourly total   MAYBE: Adiabatic tendency of temperature?
    forcingPEVAP = np.array(g[9].values.reshape(nrows*ncols)[xy_list])
        
    # 10:61:61 APCP, Precipitation hourly total [kg/m^2/hr]
    pptrate = np.array(g[10].values.reshape(nrows*ncols)[xy_list]) / 60 / 60
        
    # 11:204:204 DSWRF, Shortwave radiation flux downwards (surface) [W/m^2]
    SWRadAtm = np.array(g[11].values.reshape(nrows*ncols)[xy_list])
                 
    G={'airtemp':airtemp, 'spechum':spechum, 'airpres':airpres, 'forcingUGRD':forcingUGRD, 
       'windspd':windspd, 'LWRadAtm':LWRadAtm, 'forcingCONVfrac':forcingCONVfrac, 
       'forcingCAPE':forcingCAPE,'forcingPEVAP':forcingPEVAP, 'pptrate':pptrate, 'SWRadAtm':SWRadAtm}
           
    if verbose:
        print("TMP, 2-m above ground Temperature [K]: {}".format(airtemp))
        print("SPFH, 2-m above ground Specific humidity [kg/kg]: {}".format(spechum))
        print("PRES, Surface pressure [Pa]: {}".format(airpres))
        print("UGRD, 10-m above ground Zonal wind speed [m/s]: {}".format(forcingUGRD))
        print("VGRD, 10-m above ground Meridonal wind speed [m/s]: {}".format(windspd))
        print("DLWRF,  Longwave radiation flux downwards (surface) [W/m^2]: {}".format(LWRadAtm))
        print("CONVfrac, Fraction of total precipitation that is convective: {}".format(forcingCONVfrac))
        print("CAPE, 180-mb above ground Convective Available Potential Energy: {}".format(forcingCAPE))
        print("PEVAP, Potential evaporation hourly total: {}".format(forcingPEVAP))
        print("APCP, Precipitation hourly total [kg/m^2/hr]: {}".format(pptrate))
        print("DSWRF, Shortwave radiation flux downwards (surface) [W/m^2]: {}".format(SWRadAtm))
            
    return G


In [130]:
# Save the forcing data for each cell, individually
for ixy in tqdm(xy_list):
    xy = npt.name_xy(ixy, lats, lons)
    x, y = np.unravel_index(ixy, (ncols,nrows))
    lat=lats[ixy]
    lon=lons[ixy]
    timestp=3600 #seconds
    # Write the NetCDF forcing data file.
    fname = "{}-{}".format(lat, -lon)
    forcingDataName = write_dir + fname +'.nc'
    forcing = nc.Dataset(forcingDataName, 'w', format='NETCDF4_CLASSIC')
    forcing.title = "NLDAS forcing "+fname
    forcing.description = 'NLDAS forcing data for '+fname
    forcing = npt.fillForcing(forcing, H, lat, lon, timestp, time_series, 
        G[xy]['SWRadAtm'], 
        G[xy]['LWRadAtm'], 
        G[xy]['airpres'], 
        G[xy]['airtemp'], 
        G[xy]['pptrate'], 
        G[xy]['spechum'], 
        G[xy]['windspd'])
    os.chmod(forcingDataName, 0o777)

In [102]:
ds = nc.Dataset(forcingDataName)
print(ds)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: NLDAS forcing 25.063-124.938
    description: NLDAS forcing data for 25.063-124.938
    dimensions(sizes): hru(1), time(8747)
    variables(dimensions): float32 latitude(hr), float32 longitude(hr), int32 data_step(), float64 time(time), float32 LWRadAtm(time,hr), float32 SWRadAtm(time,hr), float32 airpres(time,hr), float32 airtemp(time,hr), float32 pptrate(time,hr), float32 spechum(time,hr), float32 windspd(time,hr)
    groups: 


In [121]:
np.array(ds.variables['SWRadAtm'][:])

array([[  0.   ],
       [  0.   ],
       [168.384],
       ...,
       [634.256],
       [492.432],
       [331.12 ]], dtype=float32)

In [125]:
npt.name_xy(0, lats, lons)

'25.063, -124.938'